In [1]:
import pandas as pd
import numpy as np

In [2]:
# data_batch = "Rent"
# block_name = "YanJiao"
file_path = f"{data_batch}_{block_name}"
df = pd.read_csv(f"{file_path}.csv")

In [4]:
# 提取所需的price和space列，并转换为数值类型
# 使用pd.to_numeric将非数值转换为NaN，然后删除包含NaN的行
processed_df = pd.DataFrame()

price = "总价" if data_batch == "Price" else "月租金"
space = "面积"

processed_df['price'] = df[price].str.extract(r'(\d+\.?\d*)', expand=False).astype(float)
processed_df['space'] = df['面积'].str.extract(r'(\d+\.?\d*)', expand=False).astype(float)
if data_batch == "Price":
    processed_df["price"] = processed_df["price"] * 10000

# 删除包含NaN值的行
processed_df = processed_df.dropna()
print(f"{block_name}地区{data_batch}共有{df.shape[0]}条数据，异常行数{df.shape[0]-processed_df.shape[0]}条")

YanJiao地区Rent数据异常行数：0


In [5]:
# 计算solo_price（price/space）并保留两位小数
processed_df['solo_price'] = (processed_df['price'] / processed_df['space']).round(2)

# 计算z-score（该值距离平均值为几个标准差）
mean_solo = processed_df['solo_price'].mean()
std_solo = processed_df['solo_price'].std()

# 避免除零错误
if std_solo == 0:
    print("数据标准差为0，请检查数据。")
    processed_df['z_score'] = 0
else:
    processed_df['z_score'] = (processed_df['solo_price'] - mean_solo) / std_solo

In [6]:
# 添加异常值标注列OOD
processed_df['OOD'] = '-'

# 根据z-score的绝对值标注异常点
mask_3sigma = np.abs(processed_df['z_score']) > 3
mask_2sigma = (np.abs(processed_df['z_score']) > 2) & (np.abs(processed_df['z_score']) <= 3)

processed_df.loc[mask_2sigma, 'OOD'] = '2sigma异常点'
processed_df.loc[mask_3sigma, 'OOD'] = '3sigma异常点'

In [9]:
OOD_2 = (processed_df["OOD"] == '2sigma异常点').sum()
OOD_3 = (processed_df["OOD"] == '3sigma异常点').sum()

# print(f"{block_name}地区{data_batch}数据共有{processed_df.shape[0]}条")
print(f"其中有{OOD_2}个2sigma异常点，{OOD_3}个3sigma异常点")

YanJiao地区Rent数据共有205条
其中有0个2sigma异常点，7个3sigma异常点
